In [4]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [7]:
%%R
setwd('./')

In [9]:
import pandas as pd

In [5]:
%%R

library(prolfqua)
library(tidyverse)

setwd('./')
fileData <- read.delim("proteinGroups.txt", sep = '\t')
annotation <- read.delim("prolfqua-inputAnnotation.txt", sep = '\t')   #根据startdata里的raw.file列来确定annotation的

startdata <- prolfqua::tidyMQ_ProteinGroups(fileData)
annotation <- annotation %>% mutate(raw.file = tolower(raw.file))
startdata <- dplyr::inner_join(annotation, startdata, by = "raw.file")
startdata <- dplyr::filter(startdata, nr.peptides > 1) #remove all proteins identified only by a single peptide.


#Then you need to tell prolfqua which columns in the data frame contain what information. 
#You do it using the AnalysisTableAnnotation class.
atable <- AnalysisTableAnnotation$new()
atable$fileName = "raw.file"
atable$hierarchy[["protein_Id"]] <- c("proteinID")
atable$factors[["Condition_"]] = "condition"
atable$factors[["replicate"]] = "replicate"      
atable$factorDepth <- 1
#startdata$Run_ID <- as.integer(startdata$Run_ID)
atable$setWorkIntensity("mq.protein.intensity")


config <- AnalysisConfiguration$new(atable)
adata <- setup_analysis(startdata, config)



#Create the LFQData class instance and remove zeros from data (MaxQuant encodes missing values with zero).
lfqdata <- LFQData$new(adata, config)
lfqdata$remove_small_intensities()
lfqdata$factors()

#You can convert the data into wide format.
lfqdata$to_wide()$data
wr <- lfqdata$get_Writer()
wr$write_wide(".")




## Visualization of not normalized data
lfqplotter <- lfqdata$get_Plotter()
density_nn <- lfqplotter$intensity_distribution_density()

lfqdata$get_Summariser()$plot_missingness_per_group()
lfqplotter$missigness_histogram()

stats <- lfqdata$get_Stats()
stats$violin()

prolfqua::table_facade( stats$stats_quantiles()$wide, paste0("quantile of ",stats$stat ))

stats$density_median()


## Normalize protein intensities
#We normalize the data by log2 transforming and then z−scaling.
lt <- lfqdata$get_Transformer()
transformed <- lt$log2()$robscale()$lfq
transformed$config$table$is_intensity_transformed

pl <- transformed$get_Plotter()
density_norm <- pl$intensity_distribution_density()

gridExtra::grid.arrange(density_nn, density_norm)

pl$pairs_smooth()

p <- pl$heatmap_cor()
p



## Fitting a linear model
transformed$config$table$getWorkIntensity()

formula_Condition <-  strategy_lm("transformedIntensity ~ Condition_")
# specify model definition
modelName  <- "Model"
unique(transformed$data$Condition_)
# [1] "30 ug" "10 ug"

#Contrasts <- c("30 ug - 10 ug" = "Condition_30 ug - Condition_10 ug")
#Contrasts <- c("HvsL" = "Condition_30 ug - Condition_10 ug")
#Contrasts <- c("30 ug - 10 ug" = "Condition_30ug - Condition_10ug")
#Contrasts <- c("HvsL" = "Condition_30ug - Condition_10ug")

Contrasts <- c("ups1 - ups2" = "Condition_ups1 - Condition_ups2")

#Contrasts <- c("LvsH" = "Condition_10ug - Condition_30ug")

mod <- prolfqua::build_model(
  transformed$data,
  formula_Condition,
  subject_Id = transformed$config$table$hierarchyKeys() )

mod$anova_histogram("FDR.Pr..F.")


aovtable <- mod$get_anova()
head(aovtable)
dim(aovtable)
xx <- aovtable |> dplyr::filter(FDR.Pr..F. < 0.2)
dim(xx)

signif <- transformed$get_copy()
signif$data <- signif$data |> dplyr::filter(protein_Id %in% xx$protein_Id)
hmSig <- signif$get_Plotter()$heatmap()
hmSig


## Compute contrasts
contr <- prolfqua::Contrasts$new(mod, Contrasts)
v1 <- contr$get_Plotter()$volcano()

contr <- prolfqua::ContrastsModerated$new(contr)
#contr$get_contrasts_sides()
contrdf <- contr$get_contrasts()
#View(contrdf)

plotter <- contr$get_Plotter()
v2 <- plotter$volcano()
gridExtra::grid.arrange(v1$FDR,v2$FDR, ncol = 1)

plotter$ma_plotly()


#write.csv(v1$FDR$data, "prolfqua-WaldTest.csv")
write.csv(v2$FDR$data, "prolfqua-WaldTest_moderated.csv")

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [22]:
WaldTest_moderated = pd.read_csv("./prolfqua-WaldTest_moderated.csv", header=0, sep=",", index_col=0)
WaldTest_moderated.dropna(subset=["diff", "p.value"], how="any", inplace=True)
WaldTest_moderated = WaldTest_moderated[-WaldTest_moderated["protein_Id"].str.contains("CON_")]
WaldTest_moderated = WaldTest_moderated[-WaldTest_moderated["protein_Id"].str.contains("REV_")]
positive =  WaldTest_moderated[(abs(WaldTest_moderated["diff"]) >1) &(WaldTest_moderated["FDR"] <0.05) ]
negative = WaldTest_moderated[(abs(WaldTest_moderated["diff"]) <=1) | (WaldTest_moderated["FDR"] >=0.05) ]

no_difference_ups = ["P00167", "P01133", "P02144", "P04040", "P15559", "P62937", "P63165",  "Q06830"]
difference_ups =  ["P01579", "O76070", "P63279", "P68871","P01127", "P01008", "P62988", "P10599", "P02787", "P99999", "P12081", "P51965","P10636-8", 
       "P01031", "P09211", "P02788", "P41159", "O00762", "P05413", "P00441", "P00918", "P08758", "P00915", "P01344", "P69905", 
        "P00709", "P55957", "P08263", "P61769", "P10145", "P16083", "P61626", "P02741", "P06732",
        "P01375", "P06396", "P02753", "P01112", "Q15843", "P02768"]

TP = len(positive[positive["protein_Id"].str.contains("|".join(difference_ups))])
FP = len(positive) - TP

FN = len(negative[negative["protein_Id"].str.contains("|".join(difference_ups))])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

35
4
2096
1


In [18]:
positive[-positive["protein_Id"].str.contains("UPS")]

,modelName,subject_Id,protein_Id,contrast,diff,std.error,avgAbd,statistic,df,p.value,conf.low,conf.high,sigma,FDR
100,WaldTest_moderated,sp|P00803|LEP_ECOLI,sp|P00803|LEP_ECOLI,ups1 - ups2,1.504456,0.313177,30.563330,5.292048,8.024981,0.000728,0.577849,2.431063,0.402041,0.033830
158,WaldTest_moderated,sp|P04968|ILVA_ECOLI,sp|P04968|ILVA_ECOLI,ups1 - ups2,-2.301094,0.207277,27.596230,-11.561957,8.024981,0.000003,-2.949793,-1.652396,0.281461,0.000201
767,WaldTest_moderated,sp|P0ACC3|ERPA_ECOLI,sp|P0ACC3|ERPA_ECOLI,ups1 - ups2,-1.768553,0.379101,28.459461,-5.216532,8.024981,0.000798,-2.873589,-0.663518,0.479459,0.036159
1237,WaldTest_moderated,sp|P21365|YCIC_ECOLI,sp|P21365|YCIC_ECOLI,ups1 - ups2,-2.116317,0.432666,25.303972,-5.511348,8.024981,0.000560,-3.367909,-0.864725,0.543048,0.027683
